In [ ]:
#OBS: 
# Precisa do azure configurado
# Precisa do purview configurado no azure

In [ ]:
# Importando as bibliotecas
import requests
import json
import pandas as pd
import datetime
from datetime import timedelta

from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient, AtlasEntity, AtlasProcess, TypeCategory
from pyapacheatlas.core.typedef import *
pd.options.mode.chained_assignment = None

In [ ]:
# Função para autenticar a entidade de serviço para o URL de recurso fornecido e retorna o token oauth2 de acesso
def azuread_auth(tenant_id: str, client_id: str, client_secret: str, resource_url: str):
    
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
    payload= f'grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}&resource={resource_url}'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    access_token = json.loads(response.text)['access_token']
    
    return access_token

In [ ]:
# Função para autenticar no Atlas Endpoint e retorna um objeto Client
def purview_auth(tenant_id: str, client_id: str, client_secret: str, data_catalog_name: str):
  
    oauth2 = ServicePrincipalAuthentication(
        tenant_id = tenant_id,
        client_id = client_id,
        client_secret = client_secret
    )
    client = PurviewClient(
        account_name = data_catalog_name,
        authentication = oauth2
    )
    return client

In [ ]:
# Abrir conexão com o datalake
spark.conf.set(
  "", # Endpoint do datalake
  "" # Acess Key
)

In [ ]:
#criar json com as chaves de acesso do azure e purview

# Recuperar caminho até o json de parâmetros da Autentificação do Purview
path_params = dbutils.fs.ls('''path do json no azure''')

# Recuperar dados do json de parâmetros da Autentificação do Purview
parameters = spark.read.format("json").load(path_params)

client_id = parameters.select('client_id').collect()[0][0]
client_secret = parameters.select('client_secret').collect()[0][0]
data_catalog_name = parameters.select('data_catalog_name').collect()[0][0]
resource_url = parameters.select('resource_url').collect()[0][0]
tenant_id = parameters.select('tenant_id').collect()[0][0]

#pegando data atual e formatando
data_today = datetime.datetime.today() - timedelta(hours=3, minutes=0)
data_now_str = data_today.strftime("%A %d %B %y %H:%M")
date_now = datetime.datetime.strptime(data_now_str, "%A %d %B %y %H:%M")

# Recuperar objetos de autenticação
azuread_access_token = azuread_auth(tenant_id, client_id, client_secret, resource_url)
purview_client = purview_auth(tenant_id, client_id, client_secret, data_catalog_name)

In [ ]:
def requestId(endpoint: str, url: str, response: any, payload: any, headers: any):
    endpoint = endpoint
    url = url
    payload = payload
    headers = headers
    response = response

    if(response.status_code != 200):
      print("Status:", response.status_code, "Erro no código", response.text)
    else:
      print("Status:", response.status_code, "Ok", response.text)

    return response.text

In [ ]:
#buscando dados da coleção
endpoint = "https://"+data_catalog_name+".purview.azure.com/"
url = f"{endpoint}/catalog/api/search/query?api-version=2021-05-01-preview"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {azuread_access_token}'
   }
payload = json.dumps({
        "orderby":["name"],
        "limit": 1000,
        "keywords": None,
        "filter": {
            "and": [
              {
                "or": [
                  {
                    "collectionId": "id da coleção aqui"
                  }
                ]
              },
              {
                "or": [
                  {
                    "classification": "classificação",
                    "includeSubClassifications": True
                  },
                  {
                      "classification": "classificação",
                    "includeSubClassifications": True
                  }
                ]
              }
            ]
          }
        })
response_dp = requests.post(url, headers=headers, data=payload)

requestId(endpoint, url, response_dp, headers, payload)

In [ ]:
## dataframe com todos os dados
df_value = pd.DataFrame(json.loads(response_dp.text)['value'])

## listando cada id de cada tabela
df_id = df_value['id']

## json com todas as tabelas e colunas
list_response = []
for id_ in df_id:
    endpoint = "https://"+data_catalog_name+".purview.azure.com/"
    url = f"{endpoint}/catalog/api/atlas/v2/entity/bulk?excludeRelationshipTypes=dataset_process_inputs&excludeRelationshipTypes=process_parent&excludeRelationshipTypes=direct_lineage_dataset_dataset&guid={id_}&includeTermsInMinExtInfo=false&minExtInfo=false&ignoreRelationships=false"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {azuread_access_token}'
    }
    payload = azuread_access_token

    response = requests.get(url, headers=headers, data=payload)
    requestId(endpoint, url, response, headers, payload)
    list_response.append(response.text)

In [ ]:
# dataframe com todos os dados
df_list = pd.DataFrame(list_response)

## percorrendo e listando cada coluna no dataframe para buscar nomes e classificações

df_list_res_referredEntities = pd.DataFrame()

for x in list_response:
    df_list = pd.DataFrame(json.loads(x)['referredEntities']).reindex(['attributes','classifications']).transpose().dropna()
    df_list_res_referredEntities = pd.concat([df_list_res_referredEntities, df_list])

df_list_res_referredEntities.reset_index(inplace=True)
df_list_res_referredEntities = df_list_res_referredEntities.rename(columns = {'index':'IDs'})

## listando dataframe com nome das colunas e criando nova coluna para filtrar as classificações
df_columns = df_list_res_referredEntities
df_columns['classificação'] = ''
df_columns['data'] = date_now

for i in range(df_columns.shape[0]):
    df_columns.loc[i,'attributes'] = df_columns.loc[i,'attributes']['name']
    df_columns.loc[:,'classificação'].iloc[i] = df_columns.classifications.iloc[i][0]['typeName']  

## dataframe com colunas e suas classificações
df_col = df_columns.loc[:, ['IDs','attributes', 'classificação', 'data']]

## dataframe com tabelas e suas classificações
df_tables = df_value.loc[:, ['id','name', 'classification']]

In [ ]:
# dataframe com nome das tabelas e filtrando as classificações para nova tabela
df_tables['classificação2'] = ''
df_tables['data2'] = date_now

for counter, list_ in enumerate(df_tables['classification']):
    if('nome da classificação aqui' in list_ and 'nome da classificação aqui' in list_):
        df_tables.loc[counter,
                      'nome da classificação aqui'] = 'nome da classificação aqui'
    elif 'nome da classificação aqui' in list_:
        df_tables.loc[counter,
                      'nome da classificação aqui'] = 'nome da classificação aqui'
    elif 'nome da classificação aqui' in list_:
        df_tables[counter, 'nome da classificação aqui'] = 'nome da classificação aqui'

## dataframe com tabelas e suas classificações
df_tab = df_tables.loc[:, ['id', 'name', 'classificação2', 'data2']].replace('', 'Tabela não classificada')

# 1 - criando dataframe
# 2 - trazendo id, nome e classificação das tabelas
# 3 - criando nova coluna com o id das colunas
# 4 - igualando o id da tabela com a coluna para fazer o merge

df_tab.loc[:,'Id_simplificado'] = ''
for count, item in enumerate(df_tab.id):
    df_tab.loc[:,'Id_simplificado'].iloc[count] = item[:33]

df_col.loc[:,'Id_simplificado'] = ''    
for count, item in enumerate(df_col.IDs):
    df_col.loc[:,'Id_simplificado'].iloc[count] = item[:33]    
    
## dataframe final com todas as tabelas e suas colunas com id, nome e classificação
tb_merge = df_tab.merge(df_col, left_on='Id_simplificado', right_on='Id_simplificado')

## dataframe final com tabelas e colunas classificadas
tb_data = tb_merge[['name', 'classificação2', 'attributes', 'classificação', 'data']].rename(columns={'name':'novo nome', 'classificação2':'novo nome', 'attributes':'novo nome', 'classificação':'novo nome', 'data': 'novo nome'})

In [ ]:
# tabela final com tabelas e colunas com apenas classificação
df = tb_data.loc[((tb_data['novo nome'] == 'sua classificação aqui') | (
    tb_data['novo nome'] == 'sua classificação aqui'))]
df


In [ ]:
# Função para exportar na pasta temporária databricks e depois mover para pasta no azure
def export_file(dataframe, ext_target, target, new_name):
  df_res = dataframe
  df_res.to_csv('/dbfs/tmp/nome da pasta/'+new_name+'.' +
                ext_target, sep=',', index=False)
  dbutils.fs.mv("/tmp/nome da pasta/"+new_name+'.'+ext_target,
                target+'/'+new_name+'.'+ext_target)
  print('Novo arquivo {} salvo em {}'.format(new_name, target))


In [ ]:
# Criar arquivo json ou txt em csv no container landing
dataframe = df_list
ext_target = "csv"
path_raw = 'path do landing no azure'
new_name = f"nome do arquivo"
export_file(dataframe, ext_target, path_raw, new_name)

In [ ]:
# Criar arquivo csv no container raw
dataframe = df
ext_target = "csv"
path_raw = 'path do raw no azure'
new_name = f"nome do arquivo"
export_file(dataframe, ext_target, path_raw, new_name)